In [1]:
import pandas as pd
import geopandas

import csv
import requests
import time

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
# Read station shapefile into memory
# to create geodataframe

stations = geopandas.read_file('tm_rail_stops/tm_rail_stops.shp')

In [3]:
# Query geodataframe to produce result set
# of stations of TYPE 'MAX'

stations = stations[stations.TYPE == 'MAX']

# Drop dupes

stations.drop_duplicates(subset = 'STATION',
                         keep = 'last',
                         inplace = True)

# Append 'MAX Station' to each MAX stop name
# in order for Google's Geocoding API to
# better recognize and geocode each row

stations['STATION'] = stations['STATION'].astype(str) + ' MAX Station'

# Drop every column except STATION

stations.drop(['TYPE', 'LINE', 'STATUS', 'geometry'],
              axis = 1,
              inplace = True)

# Export as headerless (and indexless) csv,
# which is required for the geocoding process below

stations.to_csv('station_names.csv',
                header = None,
                index = False)

In [4]:
# importing csv module

GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json?key=[API_KEY]'

infile = "station_names.csv"
outfile = "stations_geocoded.csv"
rows = []

# reading csv file

with open(infile, 'r') as csvfile:
    
    # creating a csv reader object
    
    csvreader = csv.reader(csvfile)

    # extracting each data row one by one
    
    for row in csvreader:
        rows.append(row)

    # get total number of rows
    
    print("Total no. of rows: %d"%(csvreader.line_num))

for row in rows:
    
    address = row[-1]
    
    print("Processing: " + address)

    params = {
                'address': address,
                'sensor': 'false',
                'region': 'USA'
            }

    # Do the request and get the response data
    
    req = requests.get(GOOGLE_MAPS_API_URL, params=params)
    res = req.json()

    # Use the first result
    
    if(len(res['results']) > 0):

        result = res['results'][0]

        geodata = dict()
        geodata['lat'] = result['geometry']['location']['lat']
        geodata['lng'] = result['geometry']['location']['lng']

        # check if the coordinates were found
        
        if(geodata['lat'] is not None and geodata['lng'] is not None):
            
            row.append(geodata['lat'])
            row.append(geodata['lng'])

    else:
        
        print("Some problems with finding the coordinates. ", res);

with open(outfile, 'w') as writeFile:
    
    writer = csv.writer(writeFile)
    writer.writerows(rows)

csvfile.close()
writeFile.close()

Total no. of rows: 97
Processing: Kings Hill/SW Salmon St MAX Station
Processing: Interstate/Rose Quarter MAX Station
Processing: N Lombard TC MAX Station
Processing: N Killingsworth St MAX Station
Processing: Providence Park MAX Station
Processing: Clinton/SE 12th Ave MAX Station
Processing: SE 17th Ave & Rhine St MAX Station
Processing: Beaverton Creek MAX Station
Processing: Delta Park/Vanport MAX Station
Processing: PSU South/SW 5th and Jackson MAX Station
Processing: PSU Urban Center/SW 6th and Montgomery MAX Station
Processing: SE Holgate Blvd MAX Station
Processing: Lents/SE Foster Rd MAX Station
Processing: Kenton/N Denver Ave MAX Station
Processing: SE Flavel St MAX Station
Processing: SE Powell Blvd MAX Station
Processing: SE Division St MAX Station
Processing: SE Main St MAX Station
Processing: Galleria/SW 10th Ave MAX Station
Processing: Pioneer Square North MAX Station
Processing: Parkrose/Sumner TC MAX Station
Processing: Sunset TC MAX Station
Processing: Mt Hood Ave MAX 